In [1]:
import pandas as pd 

df = pd.read_csv('nifty.csv')
df['Date'] = df['HistoricalDate']
df = df.sort_values('Date')
df  = df.reset_index()
df['price'] = df.CLOSE
df['prevPrice'] = df.price.shift(1)


In [2]:
df.head()

,index,Unnamed: 0,Index Name,INDEX_NAME,HistoricalDate,OPEN,HIGH,LOW,CLOSE,Date,price,prevPrice
0,2043,2043,Nifty 100,Nifty 100,2015-01-01,8280.55,8305.60,8259.50,8295.5,2015-01-01,8295.5,NaN
1,2042,2042,Nifty 100,Nifty 100,2015-01-02,8300.45,8416.45,8300.45,8399.9,2015-01-02,8399.9,8295.5
2,2041,2041,Nifty 100,Nifty 100,2015-01-05,8411.75,8453.35,8370.80,8383.3,2015-01-05,8383.3,8399.9
3,2040,2040,Nifty 100,Nifty 100,2015-01-06,8331.65,8331.65,8119.40,8133.9,2015-01-06,8133.9,8383.3
4,2039,2039,Nifty 100,Nifty 100,2015-01-07,8127.35,8160.05,8070.00,8112.3,2015-01-07,8112.3,8133.9


In [8]:
float(df.iloc[[2]]['price'])

8383.3

In [10]:
df['cur_idx'] = df.index

In [13]:
def get_momentum(row):
    lag_30_price = float(df.iloc[[row.cur_idx - 30]]['price'])
    lag_90_price = float(df.iloc[[row.cur_idx - 60]]['price'])
    lag_120_price = float(df.iloc[[row.cur_idx - 120]]['price'])
    
    cur_price = row.prevPrice
    
    if((cur_price> lag_30_price) & (cur_price > lag_90_price) & (cur_price > lag_120_price)):
        return 1
    
    if ((cur_price< lag_30_price) & (cur_price < lag_90_price) & (cur_price < lag_120_price)):
        return -1
    
    
    return 0
    

In [14]:
df['momentum_ind'] = df.apply(lambda row: get_momentum(row), axis=1)

In [17]:
len(df[df['momentum_ind'] == -1])

386

In [22]:
df = df[df['Date']>'2016-01-01']

from pyxirr import xirr
df['qty_traded'] = df['momentum_ind']
df['trading_price'] = df.qty_traded * -1.0 * df.CLOSE
df['total_qty'] = df['qty_traded'].cumsum()
df['inflow'] = df['trading_price']
df['mktval'] = df['total_qty'] * df['CLOSE']
df.at[len(df)-1, 'inflow'] = df.at[len(df)-2, 'mktval']
xirr(df.Date, df.inflow)

/var/folders/05/xdl3n7b519g2ls5rdzbb2tkm0000gn/T/ipykernel_810/2297463083.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['qty_traded'] = df['momentum_ind']
/var/folders/05/xdl3n7b519g2ls5rdzbb2tkm0000gn/T/ipykernel_810/2297463083.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trading_price'] = df.qty_traded * -1.0 * df.CLOSE
/var/folders/05/xdl3n7b519g2ls5rdzbb2tkm0000gn/T/ipykernel_810/2297463083.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

0.13250545284277152